# AMES HOUSING - Advanced Regression Techniques

## 0) Notebook Dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_log_error, 


## 1) Business Understanding

## 2) Data Understanding

## 3) Data Preparation

## 4) Data Modelling

## 5) Evaluation

## 6) Conclusion & Submissions